# 1 TensorBoard
TensorFlow 可用于训练大规模深度神经网络所需的计算，使用该工具涉及的计算往往复杂而深奥。为了更方便 TensorFlow 程序的理解、调试与优化，谷歌发布了一套名为 TensorBoard 的可视化工具。您可以用 TensorBoard 来展现 TensorFlow 图，绘制图像生成的定量指标图以及显示附加数据（如其中传递的图像）。

## 1.1 启动TensorBoard
1. 登陆虚拟机，以下命令均在虚拟机内执行
2. 停止并删除当前实验环境
```
docker stop datalab
docker rm datalab
```
3. 重新启动datalab,相比较以前的命令而言，增加了一个新的端口映射6006。
```
docker run -itd -p "0.0.0.0:8081:8080" -p "0.0.0.0:6006:6006" -v "${HOME}:/content" \
    --restart=always --name=datalab  \
    registry.cn-hangzhou.aliyuncs.com/oedu/datalab:local-20180214
```
4. 进入datalab容器shell环境，并启动TensorBoard (**后续每次要运行TensorBoard都需要执行该步骤**)
```
docker exec -it datalab /bin/sh
tensorboard -logdir /content
```
5. 在虚拟机软件中添加端口映射，将主机6007端口映射到虚拟机的6006端口

![image.png](http://p811pjpxl.bkt.clouddn.com/14-1.png)

设置完毕后，在浏览器可以访问tensorboard，效果如下图所示：
![image.png](http://p811pjpxl.bkt.clouddn.com/14-2.png)

## 1.2 小试牛刀

In [6]:
import tensorflow as tf

def add_layer(inputs, in_size, out_size, activation_function=None):
    # add one more layer and return the output of this layer
    Weights = tf.Variable(tf.random_normal([in_size, out_size]), name='W')
    biases = tf.Variable(tf.zeros([1, out_size]) + 0.1, name='b')
    Wx_plus_b = tf.add(tf.matmul(inputs, Weights), biases)
    if activation_function is None:
        outputs = Wx_plus_b
    else:
        outputs = activation_function(Wx_plus_b, )
    
    return outputs


# define placeholder for inputs to network
xs = tf.placeholder(tf.float32, [None, 1], name='x_input')
ys = tf.placeholder(tf.float32, [None, 1], name='y_input')

# add hidden layer
l1 = add_layer(xs, 1, 10, activation_function=tf.nn.relu)

# add output layer
prediction = add_layer(l1, 10, 1, activation_function=None)

# the error between prediciton and real data
loss = tf.reduce_mean(tf.reduce_sum(tf.square(ys - prediction),
                                        reduction_indices=[1]))

train_step = tf.train.GradientDescentOptimizer(0.1).minimize(loss)

with tf.Session() as sess:
    # 将当前会话中的计算图保存
    writer = tf.summary.FileWriter("logs/", sess.graph)

    init = tf.global_variables_initializer()
    sess.run(init)

运行上述代码，通过浏览器访问http://localhost:6007 
可以发现TensorBoard中显示了我们代码对应的计算图，以及计算图中的节点。
![image.png](http://p811pjpxl.bkt.clouddn.com/14-3.png)

## 1.3 变量命名空间

In [7]:
a = tf.Variable(tf.zeros([10]), name='a')
print(a)

with tf.name_scope('test'):
    a1 = tf.Variable(tf.zeros([10]), name='a')

print(a1)

<tf.Variable 'a:0' shape=(10,) dtype=float32_ref>
<tf.Variable 'test/a:0' shape=(10,) dtype=float32_ref>


使用命名空间不仅可以更好的管理变量，还可以优化TensorBoard的显示：
![image.png](http://p811pjpxl.bkt.clouddn.com/14-4.png)

练习：将1.2中的代码对应的变量放入指定的命名空间，具体要求如下(粗体为变量名，其他为命名空间名字，缩进代表命名空间层次)：

- layer
    - weights
        - **Weights**
    - biases
        - **biases**
    - Wx_plus_b
        - **Wx_plus_b**

- inputs
    - **xs**
    - **ys**
    
- loss
    - **loss**
    
- train
    - **train_step**
    
运行成功后，在TensorBoard中看到的结果如下图所示：
![image.png](http://p811pjpxl.bkt.clouddn.com/14-5.png)

In [2]:
# 练习代码......

## 1.4 计算图(tf.Graph)


**如果将1.3中的代码执行多遍，刷新TensorBoard界面，你可能会看到这样的显示，为什么呢？**
![image.png](http://p811pjpxl.bkt.clouddn.com/14-6.png)

计算图是排列成一个图的一系列 TensorFlow 指令。图由两种类型的对象组成。
- 指令（或“op"）：图的节点。 指令说明的是消耗和生成张量的计算。
- 张量：图的边。它们代表将流经图的值。大多数 TensorFlow 函数会返回 tf.Tensors。

![计算图](https://www.safaribooksonline.com/library/view/learning-tensorflow/9781491978504/assets/letf_0304.png)



关于计算图的详细介绍，请参考：https://www.tensorflow.org/programmers_guide/graphs?hl=zh-cn

In [5]:
a = tf.constant(5) 
print(a)
print(a.graph)
print(tf.get_default_graph())

with tf.Graph().as_default() as g:
    b = tf.constant(2)
    print(b)
    print(b.graph)
    print(g)
    
print(tf.get_default_graph())

Tensor("Const_2:0", shape=(), dtype=int32)
Tensor("Const:0", shape=(), dtype=int32)


练习：使用`tf.Graph()`创建一个新的计算图，将1.3的代码所有的TensorFlow节点都放入到该计算图中，多次执行相关代码后在TensorBoard不会出现多张图的情形。

In [2]:
# 练习代码......

# 2. FFNN

In [3]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

# 导入数据
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

# 构建计算图
sess = tf.InteractiveSession()

X = tf.placeholder(tf.float32, [None, 784], name="X")

# L1：200个神经元
W1 = tf.Variable(tf.truncated_normal([784, 200], stddev=0.1))
B1 = tf.Variable(tf.zeros([200]))
Y1 = tf.nn.sigmoid(tf.matmul(X, W1)+B1)

# L2：100个神经元
W2 = tf.Variable(tf.truncated_normal([200, 100], stddev=0.1))
B2 = tf.Variable(tf.zeros([100]))
Y2 = tf.nn.sigmoid(tf.matmul(Y1, W2)+B2)

# L3: 60个神经元
W3 = tf.Variable(tf.truncated_normal([100, 60], stddev=0.1))
B3 = tf.Variable(tf.zeros([60]))
Y3 = tf.nn.sigmoid(tf.matmul(Y2, W3)+B3)

# L4: 30个神经元
W4 = tf.Variable(tf.truncated_normal([60, 30], stddev=0.1))
B4 = tf.Variable(tf.zeros([30]))
Y4 = tf.nn.sigmoid(tf.matmul(Y3, W4)+B4)

# L5: 10个神经元
W5 = tf.Variable(tf.truncated_normal([30, 10], stddev=0.1))
B5 = tf.Variable(tf.ones([10]))
Ylogits = tf.matmul(Y4, W5) + B5

# Output
Y = tf.nn.softmax(Ylogits)

# Y_: 标签
Y_ = tf.placeholder(tf.float32, [None, 10])

# 损失函数
cross_entropy = tf.reduce_mean(-tf.reduce_sum(Y_ * tf.log(Y), reduction_indices=[1]))

# 优化算法
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)


tf.global_variables_initializer().run()

for i in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    _ = sess.run([train_step], feed_dict={X: batch_xs, Y_: batch_ys})

    if i%100 == 0:
        # 计算准确率
        correct_prediction = tf.equal(tf.argmax(Y, 1), tf.argmax(Y_, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        print(accuracy.eval({X: mnist.test.images, Y_: mnist.test.labels}))
        
sess.close()

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
0.0982
0.1009
0.1135
0.1028
0.1009
0.1177
0.1965
0.1596
0.2489
0.2659


## 2.2 重构

练习：上述代码中在定义隐藏层时存在大量重复代码，
1. 请使用1.2中的`add_layer`函数重构2.1中的代码;
2. 使用命名空间，将相应的代码加入到指定的命名空间;    
3. 使用`tf.Graph()`，将所有的代码加入到新创建的计算图中；
4. 将对应的计算图输出，通过TensorBoard显示；

命名空间如下所示：
- input
    - **X**
    - **Y_**
- loss
    - **cross_entropy**
- train
    - **train**
    
TensorBoard显示结果如下图所示：
![image.png](http://p811pjpxl.bkt.clouddn.com/14-7.png)

In [6]:
##练习代码...

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
0.0966
0.1028
0.1028
0.1029
0.1036
0.1161
0.2598
0.5097
0.6092
0.6578


## 2.3 研究

基于2.2的成果，请查阅资料在TensorBoard中将损失函数的值(`cross_entropy`)显示出来。

参考：https://morvanzhou.github.io/tutorials/machine-learning/tensorflow/4-2-tensorboard2/

![image.png](http://p811pjpxl.bkt.clouddn.com/14-8.png)

# 参考
1. https://morvanzhou.github.io/tutorials/machine-learning/tensorflow/4-1-tensorboard1/